### Optimization using skopt

In [1]:
import os
import numpy as np
import json

import sys
sys.path.append('./..')

from pythiamill import PythiaMill
from pythiamill.utils import TuneMCDetector
import time

from skopt import Optimizer
from skopt.space import Real

from tqdm import tqdm, tqdm_notebook


Reading parameters' description.

In [2]:
with open('params/all_pars_dict.json', 'r') as iofile:
    all_param_info = json.load(iofile)

Here active blocks are selected. No more bool values, just provide target block numbers.

In [3]:
active_blocks = [3]

active_param_list = []
for block_number in active_blocks:
    for par in all_param_info:
        if int(all_param_info[par]['block'])==block_number:
            active_param_list.append(par)

Get target parameters constraints. Now only `float` values are used, so other types will raise `ValueError`.

In [4]:
active_param_info_dict = dict()
for par in active_param_list:
    par_info = {
        'type': 'FLOAT',
        'size': 1,
        'min': all_param_info[par]['min'],
        'max':all_param_info[par]['max']
    }
    
    active_param_info_dict[par]=par_info

In [5]:
active_param_info_dict

{'decupletSup': {'max': 1.0, 'min': 0.001, 'size': 1, 'type': 'FLOAT'},
 'etaPrimeSup': {'max': 1.0, 'min': 0.001, 'size': 1, 'type': 'FLOAT'},
 'etaSup': {'max': 1.0, 'min': 0.001, 'size': 1, 'type': 'FLOAT'},
 'mesonBvector': {'max': 3.0, 'min': 0.001, 'size': 1, 'type': 'FLOAT'},
 'mesonCvector': {'max': 3.0, 'min': 0.001, 'size': 1, 'type': 'FLOAT'},
 'mesonSvector': {'max': 3.0, 'min': 0.001, 'size': 1, 'type': 'FLOAT'},
 'mesonUDvector': {'max': 3.0, 'min': 0.001, 'size': 1, 'type': 'FLOAT'},
 'probQQ1toQQ0': {'max': 1.0, 'min': 0.001, 'size': 1, 'type': 'FLOAT'},
 'probQQtoQ': {'max': 1.0, 'min': 0.001, 'size': 1, 'type': 'FLOAT'},
 'probSQtoQQ': {'max': 1.0, 'min': 0.001, 'size': 1, 'type': 'FLOAT'},
 'probStoUD': {'max': 1.0, 'min': 0.001, 'size': 1, 'type': 'FLOAT'}}

For optimization process `skopt` needs dimensions in special format. Next function generates it from the info dict.

In [6]:
def extract_features_space(variables_dict):
    space = []
    for var_name, params in variables_dict.items():
        if params['type'] == 'FLOAT':
            space.append(Real(name=var_name, low=params[u'min'], high=params[u'max']))
        else:
            raise ValueError
    return space

dimensions = extract_features_space(active_param_info_dict)

Pythia needs some options to run the experiment. These options are the same as were used in TuneMC experiment. 

In [7]:
with open('params/pythia_params_prefix.json', 'r') as iofile:
    pythia_params_prefix = json.load(iofile)

Now let's get the reference solution based on Monash 13 experiment data (https://arxiv.org/pdf/1404.5630.pdf page 41). These values are already in `all_param_info` dict with key `Monash`. 

Options for sampling are combined from prefix mentioned above and selected parameters (block 1, 2 or 3) values. 

In [8]:
reference_solution = {
    x: '{}:{} = {}'.format(y['category'], x, y['Monash']) for x, y in all_param_info.items()
}

reference_options = []
reference_options.extend(pythia_params_prefix)
for par in active_param_list:
    reference_options.append(reference_solution[par])


In [9]:
reference_options

['Tune:ee = 7',
 'Beams:idA = 11',
 'Beams:idB = -11',
 'Beams:eCM = 91.2',
 'WeakSingleBoson:ffbar2gmZ = on',
 '23:onMode = off',
 '23:onIfMatch = 1 -1',
 '23:onIfMatch = 2 -2',
 '23:onIfMatch = 3 -3',
 '23:onIfMatch = 4 -4',
 '23:onIfMatch = 5 -5',
 'StringFlav:decupletSup = 1.0',
 'StringFlav:etaPrimeSup = 1.0',
 'StringFlav:etaSup = 0.6',
 'StringFlav:mesonBvector = 2.2',
 'StringFlav:mesonCvector = 0.88',
 'StringFlav:mesonSvector = 0.55',
 'StringFlav:mesonUDvector = 0.5',
 'StringFlav:probQQ1toQQ0 = 0.0275',
 'StringFlav:probQQtoQ = 0.081',
 'StringFlav:probSQtoQQ = 0.915',
 'StringFlav:probStoUD = 0.217']

In [10]:
detector = TuneMCDetector()
BATCH_SIZE = 1
N_WORKERS = 1

In [11]:
def generate_input_for_mill(tuned_params):
    global pythia_params_prefix
    global dimensions

    params = []
    params.extend(pythia_params_prefix)
    for idx, dim in enumerate(dimensions):
        param_string = "{}:{} = {}".format(all_param_info[dim.name]['category'], dim.name, tuned_params[idx])
        params.append(param_string)
    return params
        

In [12]:
def sample_points_with_params(options, n_batches=1):
    global detector, BATCH_SIZE, N_WORKERS
    mill = PythiaMill(detector, options, cache_size=16, batch_size=BATCH_SIZE, n_workers=N_WORKERS, seed=123)

#     data = np.vstack([
#         mill.sample() for _ in tqdm_notebook(range(n_batches), leave=False)
#     ])
    data = np.vstack([
        mill.sample() for _ in range(n_batches)
    ])

    mill.terminate()
    return data

Axis 0 is sample number within batch.

In [13]:
reference_solution = np.mean(sample_points_with_params(reference_options, 100), axis=0)

Instead of $\chi^2$ using JS-divergence

In [14]:
from scipy.stats import chisquare
from scipy.stats import entropy
from numpy.linalg import norm

def JSD(P, Q):
    _P = P / norm(P, ord=1)
    _Q = Q / norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))

Target function should be in certain format to work with `skopt` class `Optimizer`.

In [15]:
def wrapped_mill_quality_func(tuned_params):
    global reference_solution
    global tqdm_object

    options = generate_input_for_mill(tuned_params)
    
    s_time = time.time()
    generated = np.mean(sample_points_with_params(options, 10), axis=0)
    jsd = JSD(generated, reference_solution)
    e_time = time.time()
    tqdm_object.update(1)
    tqdm_object.set_postfix(current_jsd=jsd, elapsed_time='{:.2f} s'.format(e_time - s_time))
    return jsd


In [16]:
N_INITIAL_POINTS = 15
N_STEPS = 25

In [18]:
tqdm_object = tqdm_notebook(total=N_STEPS)
tqdm_object.set_description(desc='Total:')
opt = Optimizer(dimensions=dimensions, n_initial_points=N_INITIAL_POINTS)
res = opt.run(wrapped_mill_quality_func, N_STEPS)


HBox(children=(IntProgress(value=0, max=25), HTML(value='')))

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/local/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/local/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/local/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/local/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/local/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/local/lib/python3.6/multiprocessing/connection.py", 